# FITS Image Quality Analysis
The FITS image contains image and header data. This notebook runs through a couple of examples in...

In [ ]:
#imports
from FITSImageQA.imageqa import ImageQA, QAData, QAHeader

%load_ext autoreload
%autoreload 2


## The first step is to initilize the FITSImageQA class
Can create an `ImageQA` instance, which will hold both `QAData` and `QAHeader`, or can create either of the subclasses directly

`ImageQA` will hold three primary attributes:
    * `is_corrupt` : bool, checking whether the input FITS file is corrupt
    * `qadata` : `QAData` object
    * `qahdr` : `QAHeader` object

In [ ]:
imageqa = ImageQA('../FITSImageQA_example_images/hlsp_hlf_hst_wfc3-60mas_goodss_f140w_v2.0_sci_cutout_1arcmin.fits')

In [ ]:
# check whether file is corrupt:
imageqa.is_corrupt

In [ ]:
help(imageqa.qadata)

In [ ]:
help(imageqa.qahdr)

## Analysis
some instruction on how to analyze FITS image

### Image header analysis
Tasks include:
- `check_header_fields_present` : checking whether expected header fields are present
    * checked fields can be optionally stored as an attribute
    * by default, method will only return True/False. Also have option to return a list of the missing fields, if requested (passing arg `return_missing_fields=True`)
- `check_header_fields_dtype` : check if the header fields are of the correct dtype (can check against multiple acceptable dtypes)

#### `check_header_fields_present` examples

In [ ]:
qa_header = QAHeader("../FITSImageQA_example_images/hlsp_hlf_hst_wfc3-60mas_goodss_f140w_v2.0_sci_cutout_1arcmin.fits")

# Create a list of fields that we know are present, and an additional list with a field we know is *not* present :
expected_fields = ['DATE', 'FILTER', 'CRVAL1', 'CRVAL2']
expected_fields_bad = expected_fields + ['FIELD_NOT_PRESENT']

# show expected fields attribute - it is unchanges, unless we request otherwise
print('expected_fields attribute before running the check:', qa_header.expected_fields)
print('    result of `check_header_fields_present', qa_header.check_header_fields_present(expected_fields=expected_fields))
print('expected_fields attribute after running the check:', qa_header.expected_fields)

# optionally, return the missing fields, by setting `return_missing_fields=True`
print('\nMissing field not returned:')
print('    result of `check_header_fields_present', qa_header.check_header_fields_present(expected_fields=expected_fields_bad))
print("Missing field returned:")
print('    result of `check_header_fields_present', qa_header.check_header_fields_present(expected_fields=expected_fields_bad, return_missing_fields=True))
print("Missing field returned, when no fields are missing:")
print('    result of `check_header_fields_present', qa_header.check_header_fields_present(expected_fields=expected_fields, return_missing_fields=True))

### set the locally-input expected fields as a class attribute
# show expected fields attribute, before and after storing the locally-passed set 
print('\nexpected_fields attribute before running the check:', qa_header.expected_fields)
print('    result of `check_header_fields_present', qa_header.check_header_fields_present(expected_fields=expected_fields, overwrite_attribute=True))
print('expected_fields attribute after running the check:', qa_header.expected_fields)
#print(qa_header.expected_fields)




#### `check_header_fields_dtype` examples

In [ ]:
qa_header = QAHeader("../FITSImageQA_example_images/hlsp_hlf_hst_wfc3-60mas_goodss_f140w_v2.0_sci_cutout_1arcmin.fits")

# Create a list of fields that we know are present:
expected_fields_dtype = dict(DATE = str,
                             FILTER = str, 
                             CRVAL1 = [float, str],
                             CRVAL2 = [float, str]
)
print("Here's the result of checking the header dtypes:", qa_header.check_header_fields_dtype(expected_fields_dtype=expected_fields_dtype, exit_on_fail=True))

# Add a field that we know is not present in the header
expected_fields_dtype_missing = expected_fields_dtype.copy()
expected_fields_dtype_missing.update(dict(FIELD_NOT_PRESENT=[int, float]))
print("\nHere's the result of checking the header dtypes:", qa_header.check_header_fields_dtype(expected_fields_dtype=expected_fields_dtype_missing))

# Force a failure, and show different behavior
expected_fields_dtype_bad = expected_fields_dtype_missing.copy()
expected_fields_dtype_bad.update( dict(EXPTIME=str))
print("\nHere's the result of checking the header dtypes, collecting the failures:", qa_header.check_header_fields_dtype(expected_fields_dtype=expected_fields_dtype_bad, return_incorrect_fields=True, exit_on_fail=False))

# Optionally, store the input as an attribute
print('\nexpected_fields_dtype attribute before running the check:', qa_header.expected_fields_dtype)
print('    result of checking the header dtypes:', qa_header.check_header_fields_dtype(expected_fields_dtype=expected_fields_dtype_missing, return_incorrect_fields=False, exit_on_fail=False, overwrite_attribute=True) )
print('expected_fields_dtype attribute after running the check:', qa_header.expected_fields_dtype)


In [ ]:
# Show failure, where it fails immediately
print("\nHere's the result of checking the header dtypes, crashing on failure:", qa_header.check_header_fields_dtype(expected_fields_dtype=expected_fields_dtype_bad, exit_on_fail=True))


### Image data analysis
Tasks include:
- `detect_sources` : run source detect using `sep` (via `detection.extract_sources`), and store both the catalog and corresponding segmentation map
    * can override `sep.extract` source detection parameters by passing them to `detect_sources`
- `is_focus_good` : compares the median FWHM of the source detection catalog to a canonical value. Uses the already-stored source detection catalog or implicitly generates the source detection catalog, if `detect_sources` has not been run yet
- `display_image` : plot the image, with several options:
    * overplot detected sources
    * save the image (must include file extension)
    * plot using simple matplotlib figure (no WCS axes) or using aplpy

#### `detect_sources` examples

In [ ]:
imageqa.qadata.detect_sources()
print("Get help with the result:")
help(imageqa.qadata.sources)

In [ ]:
### source detection parameters can be set by passing a dictionary to the 
###     `detection_config` (class attribute, which will eventually be set 
###     at time of instantiating the object), and by passing additional kwargs
sep_kwargs = dict(thresh=1.5, minarea=3., deblend_nthresh=64)

### Can detect sources again, but must specify overwrite = True (must forcefully clobber an existing source catalog + segmap)
imageqa.qadata.detect_sources(detection_config=sep_kwargs, deblend_cont=0.0025)

### Ultimately, we should be storing all of the relevant detection parameters in `self.detection_config`. 
#       But some are currently buried in the detection.extract_sources defaults

print("Here is the detection config [to be improved]: ", imageqa.qadata.detection_config)

#### quality checks: `is_focus_good`

In [ ]:
# Check if the image is in focus, by comparing the median FWHM to a user-specified value (default = 2.5)
is_good, med_fwhm = imageqa.qadata.is_focus_good(max_focus_fwhm=2.5)
print(is_good, med_fwhm)

## Maybe some visualization
Do we need to visualize any of the results from analysis?

In [ ]:
#Visualize some results

In [ ]:
imageqa.qadata.display_image(add_detections=False, vmin=0., vmax=0.05)